# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("AZURE_OPENAI_API_KEY")
_set_env("AZURE_OPENAI_ENDPOINT")
_set_env("AZURE_OPENAI_API_VERSION")
_set_env("AZURE_OPENAI_DEPLOYMENT_NAME")

In [2]:
llm_config: dict = {
    "config_list": [
        {
            "model": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            "api_type": "azure",
            "api_key": os.environ["AZURE_OPENAI_API_KEY"],
            "base_url": os.environ["AZURE_OPENAI_ENDPOINT"],
            "api_version": os.environ["AZURE_OPENAI_API_VERSION"],
        }
    ]
}

## The task!

In [ ]:
task = """
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       """

## Create a writer agent

In [23]:
from autogen import AssistantAgent

In [24]:
writer: AssistantAgent = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
    "blogpost (with title) on given topics. You must polish your "
    "writing based on the feedback you receive and give a refined "
    "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [25]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [26]:
print(reply)

**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of the AI revolution, empowering individuals and organizations to harness the power of artificial intelligence. Founded by AI pioneer Andrew Ng, this platform offers a comprehensive suite of resources, including courses, certification programs, and community support. With a focus on practical applications, learners can deepen their understanding of neural networks, machine learning, and data science. Whether you are a beginner eager to explore or an expert seeking to refine your skills, DeepLearning.AI provides the tools and knowledge to navigate the evolving landscape of AI and unlock new opportunities in various industries.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [28]:
from autogen.agentchat.chat import ChatResult

In [27]:
critic: AssistantAgent = AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
    "the writer and provide constructive "
    "feedback to help improve the quality of the content.",
)

In [29]:
res: ChatResult = critic.initiate_chat(
    recipient=writer, message=task, max_turns=2, summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential: DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals through innovative courses and resources. Founded by AI pioneer Andrew Ng, it offers comprehensive programs that demystify complex concepts, making AI accessible to everyone—from beginners to seasoned professionals. With hands-on projects and a focus on real-world applications, learners gain the skills necessary to thrive in this rapidly evolving field. Joining DeepLearning.AI means being part of a vibrant community dedicated to advancing AI knowledge and capabilities. Embrace the future of technology and unleash your potential in the world of artificial intelligence.

----------------------

## Nested chat

In [12]:
from autogen import AssistantAgent

In [13]:
SEO_reviewer: AssistantAgent = AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
    "your ability to optimize content for search engines, "
    "ensuring that it ranks well and attracts organic traffic. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

In [14]:
legal_reviewer: AssistantAgent = AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
    "your ability to ensure that content is legally compliant "
    "and free from any potential legal issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

In [17]:
ethics_reviewer: AssistantAgent = AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
    "your ability to ensure that content is ethically sound "
    "and free from any potential ethical issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role. ",
)

In [18]:
meta_reviewer: AssistantAgent = AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [19]:
def reflection_message(recipient, messages, sender, config):
    return f"""Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"""


review_chats: list[dict] = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'reviewer': '', 'review': ''}",
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1,
    },
]

In [20]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [21]:
res = critic.initiate_chat(
    recipient=writer, message=task, max_turns=2, summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential: DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals through innovative courses and resources. Founded by AI pioneer Andrew Ng, it offers comprehensive programs that demystify complex concepts, making AI accessible to everyone—from beginners to seasoned professionals. With hands-on projects and a focus on real-world applications, learners gain the skills necessary to thrive in this rapidly evolving field. Joining DeepLearning.AI means being part of a vibrant community dedicated to advancing AI knowledge and capabilities. Embrace the future of technology and unleash your potential in the world of artificial intelligence.

----------------------

## Get the summary

In [22]:
print(res.summary)

**DeepLearning.AI: Your Gateway to AI Education**

Discover the future of artificial intelligence with DeepLearning.AI, a leading platform delivering comprehensive AI education. Founded by AI expert Andrew Ng, it provides high-quality courses in artificial intelligence, machine learning training, and practical projects, making AI accessible to all. Emphasizing inclusivity, DeepLearning.AI ensures that learners from diverse backgrounds can enhance their skills. With a commitment to data privacy and transparency, they safeguard your personal information while nurturing your growth in this dynamic field. Join a thriving community and unlock your potential—enroll in DeepLearning.AI today! 

*Meta Description: Explore DeepLearning.AI for top-notch AI courses, fostering inclusivity and data privacy. Join the learning revolution today!*
